In [1]:
# Делал в Google Colab т.к. на macos так и не разобрался с ошибками установки спарка!!!

# Установка Java
!apt-get update
!apt-get install -y openjdk-11-jdk-headless

# Установка PySpark
!pip install pyspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,860 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,519 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-

# Задача 2: Работа с PySpark

## Импорт необходимых библиотек

In [2]:
from pyspark.sql import SparkSession

## Инициализация SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("Bike Store Analysis") \
    .getOrCreate()

## Загрузка данных в PySpark DataFrame

In [4]:
# Список файлов для импорта и их соответствующих переменных
files = {
    "products": "products.csv",
    "brands": "brands.csv",
    "categories": "categories.csv",
    "customers": "customers.csv",
    "order_items": "order_items.csv",
    "orders": "orders.csv",
    "staffs": "staffs.csv",
    "stocks": "stocks.csv",
    "stores": "stores.csv"
}

# Словарь для хранения DataFrame
dataframes = {}

# Импорт данных из каждого файла
for table_name, file_path in files.items():
    print(f"Loading data from {file_path} into table {table_name}...")
    try:
        # Чтение CSV в DataFrame
        df = spark.read.csv(file_path, header=True, inferSchema=True)
        dataframes[table_name] = df
        print(f"Data from {file_path} successfully loaded into table {table_name}.")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

# Пример вывода данных для проверки
for table_name, df in dataframes.items():
    print(f"\nPreview of {table_name}:")
    df.show(5)

Loading data from products.csv into table products...
Data from products.csv successfully loaded into table products.
Loading data from brands.csv into table brands...
Data from brands.csv successfully loaded into table brands.
Loading data from categories.csv into table categories...
Data from categories.csv successfully loaded into table categories.
Loading data from customers.csv into table customers...
Data from customers.csv successfully loaded into table customers.
Loading data from order_items.csv into table order_items...
Data from order_items.csv successfully loaded into table order_items.
Loading data from orders.csv into table orders...
Data from orders.csv successfully loaded into table orders.
Loading data from staffs.csv into table staffs...
Data from staffs.csv successfully loaded into table staffs.
Loading data from stocks.csv into table stocks...
Data from stocks.csv successfully loaded into table stocks.
Loading data from stores.csv into table stores...
Data from stor

## Запросы с использованием PySpark

In [7]:
# Регистрация DataFrame как временных таблиц
for table_name, df in dataframes.items():
    df.createOrReplaceTempView(table_name)

### 1. Расчёт общего объёма продаж по каждому продукту

In [8]:
query1 = """
SELECT
    p.product_name,
    SUM(oi.quantity * (oi.list_price - (oi.list_price * oi.discount))) AS total_sales
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_sales DESC
"""
result1 = spark.sql(query1)
print("Общий объём продаж по каждому продукту:")
result1.show()

Общий объём продаж по каждому продукту:
+--------------------+------------------+
|        product_name|       total_sales|
+--------------------+------------------+
|Trek Slash 8 27.5...| 555558.6111000001|
|Trek Conduit+ - 2016| 389248.7025000002|
|Trek Fuel EX 8 29...|368472.72939999995|
|Surly Straggler 6...| 226765.5509999999|
|Trek Domane SLR 6...|211584.61529999998|
|Surly Straggler -...|203507.62000000026|
|Trek Remedy 29 Ca...|203380.87009999997|
|Trek Powerfly 8 F...|188249.62349999996|
|Trek Madone 9.2 -...|175899.64819999994|
|Trek Silque SLR 8...|174524.73150000002|
|Trek Silque SLR 7...|163079.72819999995|
|Trek Fuel EX 9.8 ...|159052.69989999998|
|Electra Townie Or...|155169.41379999986|
|Heller Shagamaw F...|151160.88569999996|
|Trek Fuel EX 9.8 ...|       147449.7051|
|Electra Townie Or...|143393.39279999989|
| Trek Boone 7 - 2017|127609.63539999998|
|Trek Domane SL 6 ...|113644.67529999999|
|Surly Wednesday F...|112288.87709999991|
|Trek Remedy 9.8 -...|105469.8009999

### 2. Расчёт количества заказов по каждому статусу заказа

In [9]:
query2 = """
SELECT
    o.order_status,
    COUNT(*) AS order_count
FROM orders o
GROUP BY o.order_status
ORDER BY order_count DESC
"""
result2 = spark.sql(query2)
print("Количество заказов по каждому статусу:")
result2.show()

Количество заказов по каждому статусу:
+------------+-----------+
|order_status|order_count|
+------------+-----------+
|           4|       1445|
|           2|         63|
|           1|         62|
|           3|         45|
+------------+-----------+



### 3. Расчёт общей суммы продаж за каждый месяц

In [10]:
query3 = """
SELECT
    YEAR(o.order_date) AS year,
    MONTH(o.order_date) AS month,
    SUM(oi.quantity * (oi.list_price - (oi.list_price * oi.discount))) AS total_monthly_sales
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY YEAR(o.order_date), MONTH(o.order_date)
ORDER BY year, month
"""
result3 = spark.sql(query3)
print("Общая сумма продаж за каждый месяц:")
result3.show()

Общая сумма продаж за каждый месяц:
+----+-----+-------------------+
|year|month|total_monthly_sales|
+----+-----+-------------------+
|2016|    1| 215146.42409999995|
|2016|    2|        156112.3228|
|2016|    3|        180600.3285|
|2016|    4|        167144.0512|
|2016|    5|        205270.0091|
|2016|    6| 210562.12449999992|
|2016|    7| 199556.80889999992|
|2016|    8| 225657.37669999994|
|2016|    9|  273091.6097000001|
|2016|   10| 212078.08050000007|
|2016|   11| 182329.41239999997|
|2016|   12| 199829.97919999994|
|2017|    1| 285616.48399999994|
|2017|    2|  312923.7470000001|
|2017|    3|  308911.9018000001|
|2017|    4| 227290.91309999992|
|2017|    5| 268233.23789999995|
|2017|    6| 378865.65350000025|
|2017|    7| 229995.39789999998|
|2017|    8| 290553.45540000015|
+----+-----+-------------------+
only showing top 20 rows



### 4. Топ-5 клиентов, которые потратили больше всего денег

In [11]:
query4 = """
SELECT
    c.customer_id,
    CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
    SUM(oi.quantity * (oi.list_price - (oi.list_price * oi.discount))) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY total_spent DESC
LIMIT 5
"""
result4 = spark.sql(query4)
print("Топ-5 клиентов, которые потратили больше всего денег:")
result4.show()

Топ-5 клиентов, которые потратили больше всего денег:
+-----------+--------------+------------------+
|customer_id| customer_name|       total_spent|
+-----------+--------------+------------------+
|         94|Sharyn Hopkins|        34807.9392|
|         10|Pamelia Newman|        33634.2604|
|         75|   Abby Gamble|        32803.0062|
|          6|  Lyndsey Bean|32675.072499999995|
|         16|Emmitt Sanchez|31925.885699999995|
+-----------+--------------+------------------+

